# Objetivo

Avaliar se há clusters no portfólio 2. Em um segundo momento, a partir de uma clusterização global (na base toda), buscar novos clientes na base completa.

# Método

A partir do conjunto reduzido de features (47), aplicar um processo de clusterização (AgglomerativeClustering).

# Leitura dos arquivos

In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import feather


## Leitura da lista de features a excluir

In [2]:
# obtida em EDA Pandas_profiling_sample_10k_market.ipynb
exclude_df = pd.read_csv('../exclude_variables_generated_from_profiling.csv')
exclude_variables = exclude_df.values.squeeze().tolist()
len(exclude_variables)

143

## Carregamento Portfólio 1

In [3]:
p2 = pd.read_csv('../clientes2_merge.csv')
p2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566 entries, 0 to 565
Columns: 182 entries, Unnamed: 0 to qt_filiais
dtypes: bool(16), float64(144), int64(2), object(20)
memory usage: 743.0+ KB


## Redução no número de features

In [5]:
p2.drop(columns=exclude_variables, inplace= True)

In [6]:
p2.shape

(566, 39)

# Clusterização

## Dados do portfólio

Para usar nos dados do portfólio, temos que passar as strings para valor numérico.

In [7]:
# portfolio 2, reduzido (micro): p1micro
p2micro = p2.drop(columns=['Unnamed: 0', 'id'])

In [8]:
p2micro.dtypes == object

fl_matriz                                False
de_natureza_juridica                      True
sg_uf                                     True
natureza_juridica_macro                   True
de_ramo                                   True
setor                                     True
idade_empresa_anos                       False
idade_emp_cat                             True
fl_me                                    False
fl_sa                                    False
fl_mei                                   False
fl_ltda                                  False
dt_situacao                               True
fl_st_especial                           False
fl_email                                 False
fl_telefone                              False
fl_rm                                     True
nm_divisao                                True
nm_segmento                               True
fl_optante_simples                       False
fl_optante_simei                         False
sg_uf_matriz 

In [9]:
p2micro.head(3)

fl_matriz                       de_natureza_juridica sg_uf  \
0       True  AUTARQUIA ESTADUAL OU DO DISTRITO FEDERAL    AC   
1       True                                  MUNICIPIO    RN   
2       True                 SOCIEDADE SIMPLES LIMITADA    PI   

  natureza_juridica_macro                                            de_ramo  \
0   ADMINISTRACAO PUBLICA  ADMINISTRACAO PUBLICA, DEFESA E SEGURIDADE SOCIAL   
1                  OUTROS  ADMINISTRACAO PUBLICA, DEFESA E SEGURIDADE SOCIAL   
2  ENTIDADES EMPRESARIAIS                               SERVICOS DE EDUCACAO   

     setor  idade_empresa_anos idade_emp_cat  fl_me  fl_sa  ...  \
0  SERVIÇO           11.243836       10 a 15  False  False  ...   
1  SERVIÇO           43.789041          > 20  False  False  ...   
2  SERVIÇO           16.389041       15 a 20  False  False  ...   

     nm_meso_regiao           nm_micro_regiao fl_passivel_iss  qt_socios  \
0      VALE DO ACRE                RIO BRANCO            True        NaN   
1  AGRESTE POTIGUAR          AGRESTE POTIGUAR            True        NaN   
2   NORTE PIAUIENSE  BAIXO PARNAIBA PIAUIENSE            True        2.0   

   idade_media_socios  qt_socios_feminino  \
0                 NaN                 NaN   
1                 NaN                 NaN   
2                70.0                 1.0   

              de_faixa_faturamento_estimado  \
0  DE R$ 300.000.000,01 A R$ 500.000.000,00   
1  DE R$ 100.000.000,01 A R$ 300.000.000,00   
2    DE R$ 10.000.000,01 A R$ 30.000.000,00   

        de_faixa_faturamento_estimado_grupo vl_faturamento_estimado_aux  \
0  DE R$ 300.000.000,01 A R$ 500.000.000,00                 327487200.0   
1  DE R$ 100.000.000,01 A R$ 300.000.000,00                 176802910.0   
2    DE R$ 10.000.000,01 A R$ 30.000.000,00                  27818640.0   

   vl_faturamento_estimado_grupo_aux  
0                        327487200.0  
1                        176802910.0  
2                         27818640.0  

[3 rows x 37 columns]

In [10]:
# Vou começar com 4 features: de_natureza_juridica, sg_uf, de_ramo, de_nivel_atividade
categorical_cols = ['de_natureza_juridica', 'sg_uf', 'de_ramo', 'de_nivel_atividade']
for feat in categorical_cols:
    print(p2micro[feat].value_counts())
    print('\n')

SOCIEDADE EMPRESARIA LIMITADA                                             287
MUNICIPIO                                                                 110
EMPRESA INDIVIDUAL DE RESPONSABILIDADE LIMITADA DE NATUREZA EMPRESARIA     31
SOCIEDADE ANONIMA FECHADA                                                  21
ASSOCIACAO PRIVADA                                                         21
ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU DO DISTRITO FEDERAL           19
ORGAO PUBLICO DO PODER EXECUTIVO MUNICIPAL                                 12
EMPRESARIO INDIVIDUAL                                                      10
AUTARQUIA ESTADUAL OU DO DISTRITO FEDERAL                                   6
SOCIEDADE SIMPLES LIMITADA                                                  6
FUNDO PUBLICO                                                               6
SERVICO SOCIAL AUTONOMO                                                     6
COOPERATIVA                                                     

In [11]:
# Usando LabelEncoder
#https://chrisalbon.com/machine_learning/preprocessing_structured_data/convert_pandas_categorical_column_into_integers_for_scikit-learn/
from sklearn import preprocessing
# Create a label (category) encoder object
le = preprocessing.LabelEncoder()

In [12]:
# apply le on categorical feature columns
# https://towardsdatascience.com/encoding-categorical-features-21a2651a065c

#portfolio 1 com labelencoder: p1enc
p2enc=pd.DataFrame()
categorical_cols = ['de_natureza_juridica', 'sg_uf', 'de_ramo']
# %*%*%*%%*%*%*%%*%*%*%%*%*%*%%*%*%*%%*%*%*%
# IMPORTANTE aqui, só três features, para poder visualizar em um gráfico 3D.
# Também porque de_nivel_atividade tem alguns NaN, que tem que ser resolvidos depois
# %*%*%*%%*%*%*%%*%*%*%%*%*%*%%*%*%*%%*%*%*%
p2enc[categorical_cols] = p2micro[categorical_cols].apply(lambda col: le.fit_transform(col))

p2enc[categorical_cols].head(5)

de_natureza_juridica  sg_uf  de_ramo
0                     1      0        0
1                    12      4        0
2                    25      3       19
3                    12      1        0
4                    24      4       14

In [14]:
p2enc.shape

(566, 4)

In [15]:
from sklearn.cluster import AgglomerativeClustering

cluster = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='ward')
p2enc['cluster'] = cluster.fit_predict(p2enc)

In [16]:
# aqui, estou somando um valor aleatório para fins de visualização.
# Se não for feito, um grupo grande de pontos fica condensado em um único ponto no gráfico
# e perde-se a noção de cluster
p2enc['de_natureza_juridica']=p2enc['de_natureza_juridica'] + ((np.random.rand(p2enc.shape[0])-0.5) *0.5)
p2enc['de_ramo']=p2enc['de_ramo'] + ((np.random.rand(p2enc.shape[0])-0.5) *0.5)
p2enc['sg_uf']=p2enc['sg_uf'] + ((np.random.rand(p2enc.shape[0])-0.5) *0.5)
p2enc.head(5)

de_natureza_juridica     sg_uf    de_ramo  cluster
0              1.008451  0.081256   0.147849        0
1             11.807763  4.136876   0.169850        0
2             25.062626  3.028948  18.934667        3
3             11.833989  1.175544  -0.026462        0
4             24.185355  4.014764  14.103352        3

In [17]:
import plotly.express as px
fig = px.scatter_3d(p2enc, x='de_natureza_juridica', y='sg_uf', z='de_ramo', color = 'cluster')
fig.show()

## Conclusão até o momento

Clusterização parece interessante. Observar que, das três categorias avaliadas, o `de_ramo` não teve muita influência, porque está bem polarizado em uma das classes. Próximos passos: incluir mais features (sabendo que vai impedir a visualização em mais de 3 dimensões) e aplicar redução de dimensionalidade (PCA? antes ou depois da clusterização?).